In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [2]:
batch_size = 64
img_height = 299
img_width = 299

image_size = (299,299)

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        batch_size = batch_size,
        validation_split = 0.2,
        image_size = image_size,
        subset = "both",
        shuffle = True, # default but here for clarity
        seed=42,
    )

In [3]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')
data_dir = '/mnt/cub/CUB_200_2011/images'

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="both",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
)


Found 11788 files belonging to 200 classes.
Using 9431 files for training.
Using 2357 files for validation.


In [4]:
class_names = train_ds.class_names


----

In [5]:
import matplotlib.pyplot as plt

def plot_images(
    ds,
    class_names,
):
    plt.figure(figsize=(10, 10))
    for images, labels in train_ds.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")

In [6]:
# plot_images( train_ds, class_names )

---

In [7]:
dropout=0.3

shape=(img_height, img_width, 3)



i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)

x = tf.keras.applications.xception.preprocess_input(x)

core = tf.keras.applications.xception.Xception()
core = tf.keras.applications.xception.Xception(
    # input_shape=shape,
    include_top=False,
    weights='imagenet',
)
# core.trainable = False
x = core(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(len(class_names))(x)
# x = tf.keras.layers.Dropout(dropout)(x)

model = tf.keras.Model(inputs=[i], outputs=[x])

'''
model = tf.keras.Sequential([
    
    
    
    
    # tf.keras.layers.Rescaling(1./255),
    
    tf.keras.applications.Xception(
        include_top=False,
        weights='imagenet',
        pooling = 'avg',
        # trainable=False,
    ),
    

    tf.keras.layers.Dense(
        len(class_names),
        # activation = 'softmax',
    ),
    tf.keras.layers.Dropout(dropout),
    # tf.keras.layers.Activation("softmax", dtype="float32"),
])
'''



ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True,
  ),
  metrics=['accuracy'],
)

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=60
)